In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from skopt import BayesSearchCV
from skopt.space import Integer, Real

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
data = load_breast_cancer()
X, y = data.data, data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
class CustomEarlyStopper:
    def __init__(self, no_improvement_rounds=3):
        self.no_improvement_rounds = no_improvement_rounds
        self.best_score = -np.inf
        self.rounds_without_improvement = 0

    def __call__(self, result):
        current_score = result.func_vals[-1]
        if current_score > self.best_score:
            self.best_score = current_score
            self.rounds_without_improvement = 0
        else:
            self.rounds_without_improvement += 1

        if self.rounds_without_improvement >= self.no_improvement_rounds:
            return True
        return False

In [ ]:
param_dist = {
    'n_estimators': Integer(32, 128),
    'max_depth': Integer(2, 10),   
    'min_child_weight': Integer(1, 5),
    'subsample': Real(0.1, 0.5),         
    'colsample_bytree': Real(0.1, 0.9),  
    'gamma': Real(0, 0.5),
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [1, 1.5, 2, 3],
    'learning_rate': Real(0.001, 0.1),
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
model = XGBClassifier()

In [5]:
opt = BayesSearchCV(
    model,
    param_dist,
    n_iter=20,
    cv=cv,
    n_jobs=-1,
    scoring='roc_auc',
)

early_stop = CustomEarlyStopper(no_improvement_rounds=3)
opt.fit(X_train, y_train, callback=early_stop)
model = opt.best_estimator_

In [6]:
y_pred = model.predict(X_test)
print("Точність на тесті:", accuracy_score(y_test, y_pred))

Точність на тесті: 0.9824561403508771
